In [ ]:
!open .

In [1]:
import pandas as pd
import NeoQuant as nq
import requests
import math

## get all company list

In [2]:
companies = nq.get_all_company_list()
companies.index = 'A' + companies.index

## bond spread

In [3]:
# https://www.kisrating.co.kr/ratingsStatistics/statics_spread.do
expected_ratio = 7.91

## get stock count

In [ ]:
# company_name_list = ['이리츠코크렙', 'DL이앤씨', '맵스리얼티1']
company_name_list = ['신세계']

company_code_list = []
for company_name in company_name_list:
    company_code = nq.get_company_code(company_name, companies)
    if len(company_code) > 0:
        company_code_list.append(company_code)
    else:
        print('no company code with ' + company_name)
company_code_list

In [ ]:
company_code_list = companies.index

## get fnguide info

In [ ]:
import time

total = len(company_code_list)

for num, company_code in enumerate(company_code_list):
    try:
        company_name = nq.get_company_name(company_code, companies)
#         print(num, company_code, company_name)
        print(str(num + 1) + ' / ' + str(total) + " : " + company_name + ' (' + str(company_code) + ')')

        time.sleep(1)

        try:
            snapshot_tables = nq.request_fnguide_snapshot(company_code)
        except requests.exceptions.Timeout:
            time.sleep(60)
            snapshot_tables = fs_df = nq.request_fnguide_snapshot(company_code)

        if len(snapshot_tables) < 10:
            print('>>>> not much data')
            continue
        temp_df = nq.make_basic_df(company_code, snapshot_tables)
        temp_df['name'] = company_name
        if num == 0:
            basic_df = temp_df
        else:
            basic_df = pd.concat([basic_df, temp_df], sort=False)

        temp_df = nq.make_fr_df(company_code, snapshot_tables)
        if num == 0:
            fr_df = temp_df
        else:
            fr_df = pd.concat([fr_df, temp_df], sort=False)
    except ValueError:
        print('>>>> ValueError')
        continue
    except TypeError:
        print('>>>> TypeError')
        continue
    except KeyError:
        print('>>>> KeyError')
        continue
    
        
    if num == 20:
        break


## save basic dataframe to excel

In [ ]:
basic_df.to_excel(r'data/basic_data.xlsx')

## load basic dataframe from excel

In [4]:
basic_df = pd.read_excel(r'data/basic_data.xlsx')
basic_df.index = basic_df[basic_df.columns[0]]
basic_df.index.name = ''
basic_df.drop(basic_df.columns[0], axis = 1, inplace = True)

## save fr dataframe to excel

In [ ]:
fr_df.to_excel(r'data/fr_data.xlsx')

## load fr dataframe from excel

In [5]:
fr_df = pd.read_excel(r'data/fr_data.xlsx')
fr_df.index = fr_df[fr_df.columns[0]]
fr_df.index.name = ''
fr_df.drop(fr_df.columns[0], axis = 1, inplace = True)
big_col = list(fr_df.columns)
small_col = list(fr_df.iloc[0])

new_big_col = []
for num, col in enumerate(big_col):
    if 'Unnamed' in col:
        new_big_col.append(new_big_col[num-1])
    else:
        new_big_col.append(big_col[num])

fr_df.columns = [new_big_col, small_col]
fr_df = fr_df.loc[ fr_df.index.dropna() ]

## spread

In [ ]:
expected_ratio = 7.91

## calculate adequate price

In [6]:
total = len(fr_df)

for num, code in enumerate(fr_df.index):
    company_name = nq.get_company_name(code, companies)
    print(str(num + 1) + ' / ' + str(total) + " : " + company_name + ' (' + str(code) + ')')

#     print(num, code, company_name)
    for col in reversed(fr_df.columns.levels[0]):
#         print(fr_df[col]['ROE'])
        try:
            roe = pd.to_numeric(fr_df.loc[code][col]['ROE'])
            per = pd.to_numeric(fr_df.loc[code][col]['PER'])
            pbr = pd.to_numeric(fr_df.loc[code][col]['PBR'])
            asset = pd.to_numeric(fr_df.loc[code][col]['지배주주지분']) * 100000000
            debt_rate = pd.to_numeric(fr_df.loc[code][col]['부채비율'])
            keep_rate = pd.to_numeric(fr_df.loc[code][col]['유보율'])
            interest_rate = pd.to_numeric(fr_df.loc[code][col]['배당수익률'])

                
            if not math.isnan(roe) and not math.isnan(asset):
                print('standard date : ' + col)
                stock_count = basic_df.loc[code]['주식수']
                basic_df.at[code, '기준일'] = col
                basic_df.at[code, 'roe'] = roe
                basic_df.at[code, 'per'] = per
                basic_df.at[code, 'pbr'] = pbr
                basic_df.at[code, '부채비율'] = debt_rate
                basic_df.at[code, '유보율'] = keep_rate
                basic_df.at[code, '배당수익률'] = interest_rate
                
                basic_df.at[code, 'expectation'] = expected_ratio
    #             print('stock_count : ' + str(stock_count))
                high = nq.get_more_adequate_price(asset, roe, expected_ratio, stock_count, 1)
    #             print('high : ' + str(high))
                basic_df.at[code, 'high'] = high
                middle = nq.get_more_adequate_price(asset, roe, expected_ratio, stock_count, 0.9)
                basic_df.at[code, 'middle'] = middle
                low = nq.get_more_adequate_price(asset, roe, expected_ratio, stock_count, 0.8)
                basic_df.at[code, 'low'] = low
                break
        except ValueError:
            print('>>> ValueError')
            continue


basic_df['dis_rate'] = basic_df['price'] / basic_df['low']
basic_df = basic_df.sort_values(by='dis_rate', ascending=True)

1 / 2246 : 동화약품 (A000020)
standard date : 2019/12
2 / 2246 : KR모터스 (A000040)
standard date : 2019/12
3 / 2246 : 경방 (A000050)
standard date : 2019/12
4 / 2246 : 메리츠화재 (A000060)
standard date : 2019/12
5 / 2246 : 삼양홀딩스 (A000070)
standard date : 2019/12
6 / 2246 : 하이트진로 (A000080)
standard date : 2019/12
7 / 2246 : 유한양행 (A000100)
standard date : 2020/12(E)
8 / 2246 : CJ대한통운 (A000120)
standard date : 2019/12
9 / 2246 : 하이트진로홀딩스 (A000140)
standard date : 2019/12
10 / 2246 : 두산 (A000150)
standard date : 2019/12
11 / 2246 : 성창기업지주 (A000180)
standard date : 2019/12
12 / 2246 : DL (A000210)
standard date : 2019/12
13 / 2246 : 유유제약 (A000220)
standard date : 2020/12(E)
14 / 2246 : 일동홀딩스 (A000230)
standard date : 2019/12
15 / 2246 : 한국앤컴퍼니 (A000240)
standard date : 2020/12(E)
16 / 2246 : 기아자동차 (A000270)
standard date : 2019/12
17 / 2246 : 대유플러스 (A000300)
standard date : 2019/12
18 / 2246 : 노루홀딩스 (A000320)
standard date : 2019/12
19 / 2246 : 한화손해보험 (A000370)
standard date : 2019/12
20 / 2246 : 삼화페인트

standard date : 2019/12
170 / 2246 : 남양유업 (A003920)
standard date : 2019/12
171 / 2246 : 사조대림 (A003960)
standard date : 2019/12
172 / 2246 : 롯데정밀화학 (A004000)
standard date : 2019/12
173 / 2246 : 현대제철 (A004020)
standard date : 2019/12
174 / 2246 : SG세계물산 (A004060)
standard date : 2019/12
175 / 2246 : 신흥 (A004080)
standard date : 2019/12
176 / 2246 : 한국석유공업 (A004090)
standard date : 2019/12
177 / 2246 : 태양금속공업 (A004100)
standard date : 2019/12
178 / 2246 : 동방 (A004140)
standard date : 2019/12
179 / 2246 : 한솔홀딩스 (A004150)
standard date : 2020/12(E)
180 / 2246 : 신세계 (A004170)
standard date : 2019/12
181 / 2246 : 엔피씨 (A004250)
standard date : 2019/12
182 / 2246 : 남성 (A004270)
standard date : 2019/12
183 / 2246 : 현대약품 (A004310)
standard date : 2019/11
184 / 2246 : 세방 (A004360)
standard date : 2019/12
185 / 2246 : 농심 (A004370)
standard date : 2019/12
186 / 2246 : 삼익THK (A004380)
standard date : 2019/12
187 / 2246 : 서울식품공업 (A004410)
standard date : 2019/12
188 / 2246 : 송원산업 (A004430)
standard 

standard date : 2020/12(E)
327 / 2246 : 모토닉 (A009680)
standard date : 2019/12
328 / 2246 : 삼정펄프 (A009770)
standard date : 2019/12
329 / 2246 : 엔케이물산 (A009810)
standard date : 2019/12
330 / 2246 : 한화솔루션 (A009830)
standard date : 2020/12(E)
331 / 2246 : 명신산업 (A009900)
standard date : 2019/12
332 / 2246 : 영원무역홀딩스 (A009970)
standard date : 2019/12
333 / 2246 : 한국내화 (A010040)
standard date : 2019/12
334 / 2246 : 우리종금 (A010050)
standard date : 2019/12
335 / 2246 : OCI (A010060)
standard date : 2019/12
336 / 2246 : 한국프랜지공업 (A010100)
standard date : 2019/12
337 / 2246 : 엘에스일렉트릭 (A010120)
standard date : 2019/12
338 / 2246 : 고려아연 (A010130)
standard date : 2019/12
339 / 2246 : 삼성중공업 (A010140)
standard date : 2019/12
340 / 2246 : 우진아이엔에스 (A010400)
standard date : 2019/12
341 / 2246 : 한솔피엔에스 (A010420)
standard date : 2019/12
342 / 2246 : 지코 (A010580)
standard date : 2019/12
343 / 2246 : 웰바이오텍 (A010600)
standard date : 2019/12
344 / 2246 : 현대미포조선 (A010620)
standard date : 2019/12
345 / 2246 : 진양폴리우

standard date : 2019/12
491 / 2246 : 삼성엔지니어링 (A028050)
standard date : 2019/12
492 / 2246 : 동아지질 (A028100)
standard date : 2020/12(E)
493 / 2246 : 삼성물산 (A028260)
standard date : 2019/12
494 / 2246 : 팬오션 (A028670)
standard date : 2019/12
495 / 2246 : 케이씨 (A029460)
standard date : 2019/12
496 / 2246 : 신도리코 (A029530)
standard date : 2019/12
497 / 2246 : 삼성카드 (A029780)
standard date : 2019/12
498 / 2246 : 제일기획 (A030000)
standard date : 2019/12
499 / 2246 : 케이티 (A030200)
standard date : 2019/12
500 / 2246 : KTB투자증권 (A030210)
standard date : 2019/12
501 / 2246 : 교보증권 (A030610)
standard date : 2019/12
502 / 2246 : 동원수산 (A030720)
standard date : 2019/12
503 / 2246 : 비케이탑스 (A030790)
standard date : 2019/12
504 / 2246 : 신세계인터내셔날 (A031430)
standard date : 2019/12
505 / 2246 : 신세계푸드 (A031440)
standard date : 2019/12
506 / 2246 : 콤텍시스템 (A031820)
standard date : 2019/12
507 / 2246 : 롯데관광개발 (A032350)
standard date : 2019/12
508 / 2246 : 황금에스티 (A032560)
standard date : 2019/12
509 / 2246 : LG유플러스 (A03

standard date : 2017/12
648 / 2246 : CJ제일제당 (A097950)
standard date : 2019/12
649 / 2246 : 비상교육 (A100220)
standard date : 2019/12
650 / 2246 : 진양홀딩스 (A100250)
standard date : 2019/12
651 / 2246 : S&TC (A100840)
standard date : 2019/12
652 / 2246 : SBS미디어홀딩스 (A101060)
standard date : 2019/12
653 / 2246 : 인바이오젠 (A101140)
standard date : 2019/12
654 / 2246 : 해태제과식품 (A101530)
standard date : 2019/12
655 / 2246 : 동성코퍼레이션 (A102260)
standard date : 2019/12
656 / 2246 : 쌍방울 (A102280)
standard date : 2019/12
657 / 2246 : 이연제약 (A102460)
standard date : 2019/12
658 / 2246 : 풍산 (A103140)
standard date : 2019/12
659 / 2246 : 일진전기 (A103590)
standard date : 2019/12
660 / 2246 : 한국철강 (A104700)
standard date : 2019/12
661 / 2246 : KB금융 (A105560)
standard date : 2019/12
662 / 2246 : 한세실업 (A105630)
standard date : 2020/12(E)
663 / 2246 : 우진 (A105840)
standard date : 2019/12
664 / 2246 : 미원홀딩스 (A107590)
standard date : 2019/12
665 / 2246 : LG하우시스 (A108670)
standard date : 2019/12
666 / 2246 : 컨버즈 (A109070

...

standard date : 2019/12
1608 / 2246 : 한국정밀기계 (A101680)
standard date : 2019/12
1609 / 2246 : 조이맥스 (A101730)
standard date : 2019/12
1610 / 2246 : 인화정공 (A101930)
standard date : 2019/12
1611 / 2246 : 어보브반도체 (A102120)
standard date : 2019/12
1612 / 2246 : 에이치디 (A102210)
standard date : 2019/12
1613 / 2246 : 이엔에프테크놀로지 (A102710)
standard date : 2020/12(E)
1614 / 2246 : 코오롱생명과학 (A102940)
standard date : 2019/12
1615 / 2246 : 에스앤더블류 (A103230)
standard date : 2019/12
1616 / 2246 : 우양 (A103840)
standard date : 2019/12
1617 / 2246 : 대성파인텍 (A104040)
standard date : 2019/12
1618 / 2246 : NHN벅스 (A104200)
standard date : 2019/12
1619 / 2246 : 디와이피엔에프 (A104460)
standard date : 2019/12
1620 / 2246 : 티케이케미칼 (A104480)
standard date : 2019/12
1621 / 2246 : 코렌텍 (A104540)
standard date : 2019/12
1622 / 2246 : 노랑풍선 (A104620)
standard date : 2019/12
1623 / 2246 : 원익머트리얼즈 (A104830)
standard date : 2020/12(E)
1624 / 2246 : 케이엔더블유 (A105330)
standard date : 2019/12
1625 / 2246 : 트루윈 (A105550)
standard date : 20

1759 / 2246 : 뉴파워프라즈마 (A144960)
standard date : 2019/12
1760 / 2246 : 휴젤 (A145020)
standard date : 2019/12
1761 / 2246 : 마이크로프랜드 (A147760)
standard date : 2019/12
1762 / 2246 : 제룡산업 (A147830)
standard date : 2019/12
1763 / 2246 : 비디아이 (A148140)
standard date : 2019/12
1764 / 2246 : 세경하이테크 (A148150)
standard date : 2019/12
1765 / 2246 : 알엔투테크놀로지 (A148250)
standard date : 2019/12
1766 / 2246 : 아바텍 (A149950)
standard date : 2019/12
1767 / 2246 : 하이로닉 (A149980)
standard date : 2019/12
1768 / 2246 : 인트로메딕 (A150840)
standard date : 2019/12
1769 / 2246 : 파수 (A150900)
standard date : 2019/12
1770 / 2246 : KG ETS (A151860)
standard date : 2019/12
1771 / 2246 : 나노스 (A151910)
standard date : 2019/12
1772 / 2246 : 네이블커뮤니케이션즈 (A153460)
standard date : 2019/12
1773 / 2246 : 우리이앤엘 (A153490)
standard date : 2019/12
1774 / 2246 : 옵티팜 (A153710)
standard date : 2019/12
1775 / 2246 : 아시아종묘 (A154030)
standard date : 2020/09
1776 / 2246 : 솔루에타 (A154040)
standard date : 2019/12
1777 / 2246 : 와이엠씨 (A155650)
s

standard date : 2019/12
1911 / 2246 : 제테마 (A216080)
standard date : 2019/12
1912 / 2246 : 제너셈 (A217190)
standard date : 2019/12
1913 / 2246 : 넵튠 (A217270)
standard date : 2019/12
1914 / 2246 : 싸이토젠 (A217330)
standard date : 2019/12
1915 / 2246 : 에스디생명공학 (A217480)
standard date : 2019/12
1916 / 2246 : 러셀 (A217500)
standard date : 2020/12(E)
1917 / 2246 : 켐온 (A217600)
standard date : 2019/12
1918 / 2246 : 디딤 (A217620)
standard date : 2019/12
1919 / 2246 : 강스템바이오텍 (A217730)
standard date : 2019/12
1920 / 2246 : 엔에스 (A217820)
standard date : 2019/12
1921 / 2246 : 미래생명자원 (A218150)
standard date : 2019/12
1922 / 2246 : RFHIC (A218410)
standard date : 2019/12
1923 / 2246 : 타이거일렉 (A219130)
standard date : 2019/12
1924 / 2246 : 링크제니시스 (A219420)
standard date : 2019/12
1925 / 2246 : MP한강 (A219550)
standard date : 2019/12
1926 / 2246 : 지티지웰니스 (A219750)
standard date : 2019/12
1927 / 2246 : 퓨쳐켐 (A220100)
standard date : 2019/12
1928 / 2246 : 핸디소프트 (A220180)
standard date : 2019/12
1929 / 2246 : 켐트

standard date : 2019/12
2070 / 2246 : 노터스 (A278650)
standard date : 2019/12
2071 / 2246 : 미디어젠 (A279600)
standard date : 2019/12
2072 / 2246 : 레이크머티리얼즈 (A281740)
standard date : 2019/12
2073 / 2246 : 코윈테크 (A282880)
standard date : 2019/12
2074 / 2246 : 카이노스메드 (A284620)
standard date : 2019/12
2075 / 2246 : 노바텍 (A285490)
standard date : 2019/12
2076 / 2246 : 나노브릭 (A286750)
standard date : 2019/12
2077 / 2246 : 유안타제3호스팩 (A287410)
standard date : 2019/12
2078 / 2246 : 브릿지바이오 (A288330)
standard date : 2019/12
2079 / 2246 : 에스퓨얼셀 (A288620)
standard date : 2019/12
2080 / 2246 : 아이스크림에듀 (A289010)
standard date : 2019/12
2081 / 2246 : SV인베스트먼트 (A289080)
standard date : 2020/03
2082 / 2246 : 대유에이피 (A290120)
standard date : 2019/12
2083 / 2246 : 휴네시온 (A290270)
standard date : 2019/12
2084 / 2246 : 대유 (A290380)
standard date : 2019/12
2085 / 2246 : 코리아센터 (A290510)
standard date : 2020/12(E)
2086 / 2246 : 신도기연 (A290520)
standard date : 2019/12
2087 / 2246 : 디케이티 (A290550)
standard date : 2019/12
2

standard date : 2019/12
2242 / 2246 : 코오롱티슈진 (A950160)
standard date : 2019/12
2243 / 2246 : JTC (A950170)
standard date : 2020/02
2244 / 2246 : SNK (A950180)
standard date : 2021/07(E)
2245 / 2246 : 미투젠 (A950190)
standard date : 2019/12
2246 / 2246 : 소마젠 (A950200)
>>> ValueError
>>> ValueError


## make report

In [12]:
final_df = basic_df
final_df = final_df.loc[~final_df.index.str.startswith('A9')]
final_df['dis_rate'] = final_df['price'] / final_df['low']
final_df = final_df.sort_values(by='dis_rate', ascending=True)

/Users/Shared/anaconda3/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [13]:
final_df['rank'] = final_df['dis_rate'].rank()

In [14]:
final_df = final_df.loc[final_df['roe'] > 0]

## save all data

In [15]:
final_df.to_excel(r'data/s_rim_all_stock_data.xlsx')

## filtering

In [ ]:
final_df = final_df.loc[final_df['dis_rate'] > 0.1]
final_df = final_df.loc[final_df['dis_rate'] < 1]
final_df = final_df.loc[final_df['roe'] > final_df['expectation']]

In [ ]:
final_df = final_df.loc[final_df['roe'] > 10]
final_df = final_df.loc[final_df['roe'] < 100]
final_df = final_df.loc[final_df['시가총액'] > 1000 * 100000000]

In [ ]:
final_df = final_df.loc[final_df['배당수익률'] > 1]

In [ ]:
final_df = final_df.loc[~final_df['name'].str.contains('홀딩스')]

In [ ]:
final_df = final_df.loc[final_df['roe'] > 10]

In [ ]:
len(final_df)

## save port

In [ ]:
final_df.to_excel(r'data/port_data.xlsx')

## load port

In [ ]:
# port_df = pd.read_excel(r'data/port_data.xlsx')
port_df = pd.read_excel(r'data/test_port_1.xlsx')
port_df.index = port_df[port_df.columns[0]]
port_df.index.name = ''
port_df.drop(port_df.columns[0], axis = 1, inplace = True)

## get recent price and update

In [ ]:
import time

port_list = port_df.index
total = len(port_list)
for num, company_code in enumerate(port_list):
    company_name = get_company_name(company_code, companies)
    print(str(num + 1) + ' / ' + str(total) + " : " + company_name + ' (' + str(company_code) + ')')
    time.sleep(1)
    try:
        price_df = nq.request_price_list(company_code, 'day', 100)
    except requests.exceptions.Timeout:
        time.sleep(60)
        price_df = nq.request_price_list(company_code, 'day', 100)
        
    current = price_df[price_df.columns[0]][-1]
    port_df.at[company_code, 'current'] = int(current)


## 수익률 업데이트

In [ ]:
profit_rate = (port_df['current'] - port_df['price']) / port_df['price']
profit_rate = profit_rate.dropna()
profit_rate = profit_rate.map(lambda rate: str("%.2f" % round(float(rate * 100), 2)) + '%')
port_df['수익률'] = profit_rate

## 평균 수익률

In [ ]:
profit_rate = port_df['수익률'].map(lambda rate: float(rate[:-1]))
average_profit = profit_rate.sum() / len(port_df)
print('average profit : ' + "%.2f" % average_profit + '%')

## show chart

In [ ]:
company_name = '대원산업'
company_code = nq.get_company_code(company_name, companies)


price_df = nq.request_price_list(company_code, 'day', 1000)

price_high = basic_df.loc[company_code]['high']
price_middle = basic_df.loc[company_code]['middle']
price_low = basic_df.loc[company_code]['low']

price_df['price_low'] = [price_low] * len(price_df)
price_df['price_middle'] = [price_middle] * len(price_df)
price_df['price_high'] = [price_high] * len(price_df)

nq.show_price_chart(company_code, company_name, price_df)